In [26]:
# run initial imports
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from fbprophet import Prophet
from fbprophet.plot import plot_plotly

import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go

from scipy.integrate import odeint
from scipy.optimize import curve_fit

from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import widgets

import folium
from folium.plugins import FastMarkerCluster

from datetime import datetime

In [35]:
NOW = datetime.now()
input_dir = './data/'
combined = pd.read_csv(input_dir + 'web_cases_state.csv')
combined['Last Update'] = pd.to_datetime(combined['Last_Update'])
combined['Latitude'] = combined['Lat']
combined['Longitude'] = combined['Long_']
combined['Province/State'] = combined['Province_State']
combined['Country/Region'] = combined['Country_Region']
combined = combined.sort_values('Last Update').reset_index(drop=True)

#extract lat, lon for non mapped stuff
# locations = combined.groupby(['Country/Region', 'Province/State'])['Latitude', 'Longitude'].mean().reset_index()
# locations.columns = ['Country/Region', 'Province/State', 'Latitude_Lookup', 'Longitude_Lookup']

# combined = pd.merge(left=combined, right=locations, left_on=['Country/Region', 'Province/State'], right_on=['Country/Region', 'Province/State'], how='left')
# combined['Latitude'] = combined['Latitude'].fillna(combined['Latitude_Lookup'])
# combined['Longitude'] = combined['Longitude'].fillna(combined['Longitude_Lookup'])
# combined['Province/State'] = combined['Province/State'].fillna(combined['Country/Region'])
# del combined['Latitude_Lookup'] 
# del combined['Longitude_Lookup'] 
# combined = combined.fillna(0)

In [36]:
#do a little re-ordering
combined = combined[['Last Update', 'Latitude', 'Longitude', 'Country/Region', 'Province/State','Active', 'Confirmed','Recovered','Deaths']].copy().reset_index(drop=True)
#combined = combined[combined['Country/Region'] != 0]
combined

,Last Update,Latitude,Longitude,Country/Region,Province/State,Active,Confirmed,Recovered,Deaths
0,2020-03-24 13:50:47,32.3182,-86.9023,US,Alabama,0,196,0,0
1,2020-03-24 13:50:47,41.1254,-98.2681,US,Nebraska,0,64,0,0
2,2020-03-24 13:50:47,38.3135,-117.0554,US,Nevada,0,263,0,4
3,2020-03-24 13:50:47,43.4525,-71.5639,US,New Hampshire,0,101,0,1
4,2020-03-24 13:50:47,40.2989,-74.5210,US,New Jersey,0,2844,0,27
...,...,...,...,...,...,...,...,...,...
126,NaT,18.3358,-64.8963,US,Virgin Islands,0,17,0,0
127,NaT,-31.9505,115.8605,Australia,Western Australia,173,175,1,1
128,NaT,41.1129,85.2401,China,Xinjiang,0,76,73,3
129,NaT,24.9740,101.4870,China,Yunnan,2,176,172,2


In [37]:
latest = combined.sort_values('Last Update').groupby(['Latitude','Longitude']).tail(1).copy().reset_index(drop=True)
#latest['Active'] = latest['Confirmed'] - latest['Recovered'] - latest['Deaths']

In [38]:
deaths = latest[latest['Deaths']>0]
deaths = pd.DataFrame(deaths.values.repeat(deaths.Deaths, axis=0), columns=deaths.columns)

confirmed = latest[latest['Confirmed']>0]
confirmed = pd.DataFrame(confirmed.values.repeat(confirmed.Confirmed, axis=0), columns=confirmed.columns)

us_confirmed = latest[latest['Confirmed']>0]
us_confirmed = us_confirmed[us_confirmed['Country/Region'] == 'US']
us_confirmed = pd.DataFrame(us_confirmed.values.repeat(us_confirmed.Confirmed, axis=0), columns=us_confirmed.columns)

recovered = latest[latest['Recovered']>0]
recovered = pd.DataFrame(recovered.values.repeat(recovered.Recovered, axis=0), columns=recovered.columns)

active = latest[latest['Active']>0]
active = pd.DataFrame(active.values.repeat(active.Active, axis=0), columns=active.columns)

us_active = latest[latest['Active']>0]
us_active = us_active[us_active['Country/Region'] == 'US']
us_active = pd.DataFrame(us_active.values.repeat(us_active.Active, axis=0), columns=us_active.columns)

na = latest[latest['Active'] > 0]
na_active = na[na['Country/Region'] == 'US']
na_active = na_active.append(na[na['Country/Region'] == 'Mexico'])
na_active = na_active.append(na[na['Country/Region'] == 'Canada'])
na_active = pd.DataFrame(na_active.values.repeat(na_active.Active, axis=0), columns=na_active.columns)

In [39]:
folium_map = folium.Map(location=[38.826555, -100.244867],
                        zoom_start=4,
                        tiles='CartoDB dark_matter')

                             
#FastMarkerCluster(data=list(zip(deaths['Latitude'].values, deaths['Longitude'].values)), name='Reported Deaths').add_to(folium_map)
#FastMarkerCluster(data=list(zip(recovered['Latitude'].values, recovered['Longitude'].values)), name='Recovered Cases').add_to(folium_map)
#FastMarkerCluster(data=list(zip(active['Latitude'].values, active['Longitude'].values)), name='Active Cases').add_to(folium_map)
FastMarkerCluster(data=list(zip(na_active['Latitude'].values, na_active['Longitude'].values)), name='North America Active Cases').add_to(folium_map)

folium.LayerControl().add_to(folium_map)
folium_map.save('./www/mapping.html')
folium_map